In [1]:
import pandas as pd
import numpy as np
import torch
from basicsr.utils.event_utils import *
import math

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
txt_file = 'datasets/CED/HR//CED_additional_IR_filter/outdoor_shadow_1_infrared/events/000000.txt'

In [3]:
data = pd.read_csv(txt_file, delim_whitespace=True, header=None, names=['t', 'x', 'y', 'p'], dtype={'t': np.float64, 'x': np.int16, 'y': np.int16, 'p': np.int16})

In [4]:
t = torch.from_numpy(data['t'].values)
x = torch.from_numpy(data['x'].values)
y = torch.from_numpy(data['y'].values)
p = torch.from_numpy(data['p'].values)

In [5]:
print(x.dtype)
print(y.dtype)
print(p.dtype)
print(t.shape)

torch.int16
torch.int16
torch.int16
torch.Size([97])


In [6]:
voxel = events_to_voxel_torch(x, y, t, p, 5, sensor_size=(260, 346))

In [7]:
def voxel_normalization(voxel):
    """
        normalize the voxel same as https://arxiv.org/abs/1912.01584 Section 3.1
        Params:
            voxel: torch.Tensor, shape is [num_bins, H, W]
        
        return:
            normalized voxel
    """
    abs_voxel, _ = torch.sort(torch.abs(voxel).view(-1, 1).squeeze(1))
    first_non_zero_idx = torch.nonzero(abs_voxel)[0].item()
    non_zero_voxel = abs_voxel[first_non_zero_idx:]
    norm_idx = math.floor(non_zero_voxel.shape[0] * 0.98)

    ones = torch.ones_like(voxel)

    normed_voxel = torch.where(voxel >= non_zero_voxel[norm_idx], ones, voxel)
    normed_voxel = torch.where(voxel <= -non_zero_voxel[norm_idx], -ones, voxel)
    normed_voxel = torch.where(torch.abs(voxel) < non_zero_voxel[norm_idx], voxel / non_zero_voxel[norm_idx], voxel)

    return normed_voxel

In [8]:
normed_voxel = voxel_normalization(voxel)

In [9]:
np_voxel = normed_voxel.numpy()

In [10]:
np_voxel.shape

(5, 260, 346)

In [19]:
input = 'datasets/CED/CED_additional_IR_filter/outdoor_jumping_1_infrared'
from glob import glob
import os.path as osp
events_paths = sorted(glob(osp.join(input, 'events/*.txt')))

In [20]:
events_paths[0]

'datasets/CED/CED_additional_IR_filter/outdoor_jumping_1_infrared/events/000000.txt'

In [21]:
osp.splitext('datasets/CED/CED_additional_IR_filter/outdoor_jumping_1_infrared/events/000000.txt')

('datasets/CED/CED_additional_IR_filter/outdoor_jumping_1_infrared/events/000000',
 '.txt')

In [22]:
int('000001')

1

In [36]:
import os
root_path_list = []
h5_path_list = []
folder_path = 'datasets/CED'
for root, subdir, _ in os.walk(folder_path):
    if 'images' in subdir:
        root_path_list.append(root)
        out = osp.join('datasets/CED_h5', osp.basename(root) + '.h5')
        h5_path_list.append(out)

In [37]:
h5_path_list[0]

'datasets/CED_h5/simple_rabbits.h5'

In [38]:
root_path_list[0]

'datasets/CED/CED_simple/simple_rabbits'

In [41]:
pathlist = []
for root, h5 in zip(root_path_list, h5_path_list):
    pathlist.append([root, h5])

In [42]:
pathlist[0]

['datasets/CED/CED_simple/simple_rabbits', 'datasets/CED_h5/simple_rabbits.h5']

## hdf5 analysis

In [12]:
import h5py

path = 'datasets/CED_h5/calib_outdoor.h5'
file = h5py.File(path, 'r')

In [13]:
file['images/000000'].attrs['timestamp']

'1555332068.1933038'

In [ ]:
file.keys()
print(file['events'].keys())

In [1]:
pathlist = ['datasets/CED/CED_simple/simple_rabbits', 'datasets/CED_h5/simple_rabbits.h5']

In [2]:
from basicsr.utils.hdf5_util import *

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm-4.64.1-py3.7.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
write_h5_worker(pathlist)

Package simple_rabbits.h5 finished.


In [1]:
from basicsr.utils import scandir
import os
from glob import glob
import  os.path as osp

h5_path = glob(osp.join('datasets/CED_h5', '*.h5'))

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm-4.64.1-py3.7.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import h5py
file = h5py.File('datasets/CED_h5/driving_city_5.h5', 'r')
len(file['voxel'].keys())
file['voxel/004318'].attrs['is_empty']

False

In [2]:
from basicsr.utils.hdf5_util import write_h5_worker
pathlist = [
    'datasets/CED/HR/CED_driving/driving_city_sun_2',
    'datasets/CED_h5/HR/driving_city_sun_2.h5'
]
write_h5_worker(pathlist)

datasets/CED/HR/CED_driving/driving_city_sun_2/events/001765.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001766.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001767.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001768.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001769.txt is empty, will write zero voxel


KeyboardInterrupt: 

In [8]:
from basicsr.utils.hdf5_util import write_h5_worker
pathlist = [
    'datasets/CED/HR/CED_people/people_dynamic_wave',
    'datasets/CED_h5/HR/people_dynamic_wave.h5'
]
write_h5_worker(pathlist)

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Package people_dynamic_wave.h5 finished.


In [1]:
from basicsr.utils.hdf5_util import write_h5_worker
pathlist = [
    'datasets/CED/HR/CED_simple/simple_jenga_destroy',
    'datasets/CED_h5/HR/simple_jenga_destroy.h5'
]
write_h5_worker(pathlist)


/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm-4.64.1-py3.7.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Package simple_jenga_destroy.h5 finished.


In [13]:
import pandas as pd
import numpy as np

txt_file = 'datasets/CED/CED_driving/driving_city_5/events/004318.txt'
data = pd.read_csv(txt_file, delim_whitespace=True, header=None, names=['t', 'x', 'y', 'p'], dtype={'t': np.float64, 'x': np.int16, 'y': np.int16, 'p': np.int16})

In [14]:
data.empty

False

In [15]:
a = np.zeros((5, 260, 346))

In [16]:
a.shape

(5, 260, 346)

In [1]:
import h5py
root_path = 'datasets/CED_h5/HR'
from glob import glob
import os
import os.path as osp 
h5_list = glob(osp.join(root_path, "*.h5"))

In [2]:
root_path_list = []
h5_path_list = []
folder_path = 'datasets/CED/HR'
for root, subdir, _ in os.walk(folder_path):
    if 'images' in subdir:
        root_path_list.append(root)
        out = osp.join('datasets/CED_h5/HR', osp.basename(root) + '.h5')
        h5_path_list.append(out)

In [3]:
total_path = []
for root, h5 in zip(root_path_list, h5_path_list):
    total_path.append([root, h5])
print(total_path[0])

['datasets/CED/HR/CED_simple/simple_rabbits', 'datasets/CED_h5/HR/simple_rabbits.h5']


In [4]:
cnt = 0
print(len(total_path))
for f in total_path:
    len_ori = len(glob(osp.join(f[0], 'events', "*.txt")))

    file = h5py.File(f[1], 'r')
    event_num1 = len(file['voxels'].keys())
    # if 'voxels' in file.keys():
    #     event_num2 = len(file['voxels'].keys())
    # else:
    #     event_num2 = 0
    event_num = event_num1
    if len_ori != event_num:
        print(f[1], len_ori, event_num)

84


In [15]:
file = h5py.File('datasets/CED_h5/simple_fruit_fast.h5', 'r')
print(len(file['voxel'].keys()))

932


## Cp HR/timestamp.txt to LR/timestamp.txt

In [1]:
import os
import os.path as osp

In [2]:
HR_path = 'datasets/CED/HR/'
LR_path = 'datasets/CED/LRx4'
for root, subdir, files in os.walk(HR_path):
    if 'timestamp.txt' in files:
        folder = root.split(HR_path)[1]
        src = osp.join(HR_path, folder, 'timestamp.txt')
        dst = osp.join(LR_path, folder)
        os.system(f"cp -rf {src} {dst}")

## get hr voxel and use bicubic to downsample

In [1]:
import os
import os.path as osp
import numpy as np

folder_path = 'datasets/CED/LR'
root_path_list = []
h5_hr_path_list = []
h5_lr_path_list = []
for root, subdir, _ in os.walk(folder_path):
    if 'images' in subdir:
        root_path_list.append(root)
        h5_hr_path_list.append(osp.join('datasets/CED_h5/HR', osp.basename(root) + '.h5'))
        h5_lr_path_list.append(osp.join('datasets/CED_h5/LR', osp.basename(root) + '.h5'))

In [2]:
h5_hr_path_list[0]

'datasets/CED_h5/HR/simple_rabbits.h5'

In [3]:
import h5py

file = h5py.File(h5_hr_path_list[0], 'r')


In [16]:
file.attrs.keys()

<KeysViewHDF5 ['duration', 'num_events', 'num_imgs', 'sensor_resolution', 't0', 'tk']>

In [18]:
file['voxels/000000'].attrs['is_empty']

False

In [5]:
voxel_ex.shape

(5, 260, 346)

In [8]:
import sys
sys.path.append('./scripts/data_preparation')
from scripts.data_preparation.pyResize import DownSample, UpSample, imresize

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm-4.64.1-py3.7.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
voxel_lr = imresize(voxel_ex, 0.5)

In [11]:
voxel_lr.shape

(3, 130, 346)

In [9]:
import torch
voxel_tensor = torch.from_numpy(voxel_ex).unsqueeze(0)

In [14]:
lr = torch.nn.functional.interpolate(input = voxel_tensor, scale_factor = 0.5, mode = 'bicubic').squeeze(0).numpy()

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


In [15]:
lr.shape

(5, 130, 173)

## test lr h5 file

In [10]:
import h5py
from basicsr.utils.hdf5_util import *

pathlist = [
    'datasets/CED/LR/CED_simple/simple_carpet',
    'datasets/CED_h5/HR/simple_carpet.h5',
    'datasets/CED_h5/LR/simple_carpet.h5'
]

write_lr_h5_worker(pathlist)


/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


Package simple_carpet.h5 finished.


In [11]:
import h5py
file = h5py.File('datasets/CED_h5/HR/driving_city_4.h5', 'r')
file.attrs.keys()

<KeysViewHDF5 ['duration', 'num_events', 'num_imgs', 'sensor_resolution', 't0', 'tk']>

In [15]:
file['voxels/000000'].shape

(5, 260, 346)

In [13]:
file.attrs['num_events']

17514

In [12]:
import os

txt_path = '/home/kaidc/code/e2vsr/basicsr/data/meta_info/CED_tmp.txt'

with open(txt_path, 'r') as f:
    str_temp = f.read().splitlines()
    print(str_temp)

['people/people_dynamic_wave/images 759 (260,346,3)', 'indoors/indoors_foosball_2/images 269 (260,346,3)', 'simple/simple_wires_2/images 552 (260,346,3)', 'people/people_dynamic_dancing/images 1175 (260,346,3)', 'people/people_dynamic_jumping/images 792 (260,346,3)', 'simple/simple_fruit_fast/images 933 (260,346,3)', 'additional_IR_filter/outdoor_jumping_infrared_2/images 665 (260,346,3)', 'simple/simple_carpet_fast/images 602 (260,346,3)', 'people/people_dynamic_armroll/images 792 (260,346,3)', 'indoors/indoors_kitchen_2/images 635 (260,346,3)', 'people/people_dynamic_sitting/images 1075 (260,346,3)']


In [13]:
to_write = str_temp[0].split(' ')[0].split('/')[1] + ' ' + str_temp[0].split(' ')[1]
print(to_write)

people_dynamic_wave 759


In [14]:
txt_path = '/home/kaidc/code/e2vsr/basicsr/data/meta_info/CED_h5_test.txt'

with open(txt_path, 'w') as f:
    for i in range(len(str_temp)):
        to_write = str_temp[i].split(' ')[0].split('/')[1] + '.h5 ' + str_temp[i].split(' ')[1] + '\n'
        f.write(to_write)

In [25]:
txt_path = '/home/kaidc/code/e2vsr/basicsr/data/meta_info/CED_h5_train_2.txt'

with open(txt_path, 'w') as f:
    for i in range(len(str_temp)):
        f.write(str_temp[i].split(' ')[0].split('/')[1] + '.h5\n')

In [7]:
import os
import os.path as osp
file = 'basicsr/data/meta_info/CED_h5_train.txt'

keys = []
frame_num = {}
with open(file, 'r') as fin:
    for line in fin:
        name, num = line.split(' ')
        keys.extend([f'{name}/{i:06d}' for i in range(int(num))])
        print(osp.dirname("simple_flowers_infrared.h5"))
        frame_num[osp.dirname(name)] = num.strip()

In [8]:
frame_num.get('simple_flowers_infrared')

In [5]:
frame_num = {}
name = 'simple'
num = '10'
frame_num[name] = num
type(frame_num)

dict

In [1]:
import h5py
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [13]:
file = h5py.File('datasets/CED_h5/HR/people_dynamic_wave.h5', 'r')

In [14]:
file.keys()

<KeysViewHDF5 ['images', 'voxels']>

In [15]:
img = file['images/000003'][:]

In [19]:
import numpy as np
np_path = 'datasets/CED/HR/CED_people/people_dynamic_wave/images/000000.png'
img_np = cv2.imread(np_path)

In [17]:
(img == img_np).all()

False

In [19]:
cv2.imwrite('tmp/img.png', img)

True

In [10]:
import lmdb
path = '/home/kaidc/data/VSR/CED_lmdb/train_sharp_with_val.lmdb'

env = lmdb.open(path)

In [11]:
txn = env.begin()

In [12]:
img2 = txn.get(str('people/people_dynamic_wave/images/000000').encode())

In [13]:
type(img2)

bytes

In [14]:
from basicsr.utils.img_util import imfrombytes
img2 = imfrombytes(img2, float32=True)

In [15]:
import numpy as np
np.max(img2)

0.99607843

In [13]:
cv2.imwrite('tmp/img2.png', img2)

True

In [17]:
import h5py

file = h5py.File('datasets/CED_h5/Voxel_3/LRx4/driving_city_sun_2.h5', 'r')

In [18]:
file.keys()

<KeysViewHDF5 ['images', 'voxels']>

In [19]:
file.attrs.keys()

<KeysViewHDF5 ['duration', 'num_events', 'num_imgs', 'sensor_resolution', 't0', 'tk']>

In [20]:
file['voxels/003940'].shape

(3, 65, 86)

In [21]:
file['images/000000'].shape

(65, 86, 3)

In [18]:
from basicsr.utils.hdf5_util import write_h5_worker

In [19]:
pathlist = [
    'datasets/CED/HR/CED_driving/driving_city_sun_2',
    'datasets/CED_h5/Voxel_3/HR/driving_city_sun_2.h5'
]

In [20]:
write_h5_worker(pathlist)

datasets/CED/HR/CED_driving/driving_city_sun_2/events/001765.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001766.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001767.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001768.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001769.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/003940.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/003941.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/003942.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/003943.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/005127.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/dr

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

# test mutual attention

In [1]:
import torch
from einops import rearrange
import numbers
from torch import nn as nn

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def to_3d(x):
    return rearrange(x, 'b c h w -> b (h w) c')

def to_4d(x,h,w):
    return rearrange(x, 'b (h w) c -> b c h w',h=h,w=w)

class BiasFree_LayerNorm(nn.Module):
    def __init__(self, normalized_shape):
        super(BiasFree_LayerNorm, self).__init__()
        if isinstance(normalized_shape, numbers.Integral):
            normalized_shape = (normalized_shape,)
        normalized_shape = torch.Size(normalized_shape)

        assert len(normalized_shape) == 1

        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.normalized_shape = normalized_shape

    def forward(self, x):
        sigma = x.var(-1, keepdim=True, unbiased=False)
        return x / torch.sqrt(sigma+1e-5) * self.weight

class WithBias_LayerNorm(nn.Module):
    def __init__(self, normalized_shape):
        super(WithBias_LayerNorm, self).__init__()
        if isinstance(normalized_shape, numbers.Integral):
            normalized_shape = (normalized_shape,)
        normalized_shape = torch.Size(normalized_shape)

        assert len(normalized_shape) == 1

        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.normalized_shape = normalized_shape

    def forward(self, x):
        mu = x.mean(-1, keepdim=True)
        sigma = x.var(-1, keepdim=True, unbiased=False)
        return (x - mu) / torch.sqrt(sigma+1e-5) * self.weight + self.bias

class LayerNorm(nn.Module):
    def __init__(self, dim, LayerNorm_type):
        super(LayerNorm, self).__init__()
        if LayerNorm_type =='BiasFree':
            self.body = BiasFree_LayerNorm(dim)
        else:
            self.body = WithBias_LayerNorm(dim)

    def forward(self, x):
        h, w = x.shape[-2:]
        return to_4d(self.body(to_3d(x)), h, w)


class Mutual_Attention(nn.Module):
    def __init__(self, dim, num_heads, bias):
        super(Mutual_Attention, self).__init__()
        self.num_heads = num_heads
        self.temperature = nn.Parameter(torch.ones(num_heads, 1, 1))

        self.q = nn.Conv2d(dim, dim, kernel_size=1, bias=bias)
        self.k = nn.Conv2d(dim, dim, kernel_size=1, bias=bias)
        self.v = nn.Conv2d(dim, dim, kernel_size=1, bias=bias)

        self.project_out = nn.Conv2d(dim, dim, kernel_size=1, bias=bias)


    def forward(self, x, y):

        assert x.shape == y.shape, 'The shape of feature maps from image and event branch are not equal!'

        b,c,h,w = x.shape

        q = self.q(x) # image
        k = self.k(y) # event
        v = self.v(y) # event

        q = rearrange(q, 'b (head c) h w -> b head c (h w)', head=self.num_heads)
        k = rearrange(k, 'b (head c) h w -> b head c (h w)', head=self.num_heads)
        v = rearrange(v, 'b (head c) h w -> b head c (h w)', head=self.num_heads)

        q = torch.nn.functional.normalize(q, dim=-1)
        k = torch.nn.functional.normalize(k, dim=-1)

        attn = (q @ k.transpose(-2, -1)) * self.temperature
        attn = attn.softmax(dim=-1)
        out = (attn @ v)
        out = rearrange(out, 'b head c (h w) -> b (head c) h w', head=self.num_heads, h=h, w=w)
        out = self.project_out(out)
        return out

In [1]:
import h5py

path = 'datasets/CED_h5/Voxel_3/LRx4/driving_city_sun_2.h5'

file = h5py.File(path, 'r')


In [5]:
file['voxels/000000'].attrs['is_empty']

False

In [3]:
img = torch.randn((1, 3, 64, 64))
event = torch.zeros((1, 3, 64, 64))

In [4]:
attn = Mutual_Attention(3, 1, False)

In [10]:
result = attn(img, event)

In [11]:
result

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], grad_fn=<SlowConv2DBackward0>)

In [1]:
import math

In [9]:
import torch
# attn = torch.randn((1, 1, 3, 3)).cuda()
attn = torch.zeros((1, 1, 3, 3)).cuda()
print(attn)

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]], device='cuda:0')


In [10]:
attn = attn.softmax(dim=-1)

In [11]:
attn

tensor([[[[0.3333, 0.3333, 0.3333],
          [0.3333, 0.3333, 0.3333],
          [0.3333, 0.3333, 0.3333]]]], device='cuda:0')

In [3]:
import os
import os.path as osp

In [4]:
root_path = 'datasets/CED_2/HR/'

cnt = 0
for root, subdir, _ in os.walk(root_path):
    if 'events' in subdir:
        cnt += 1
        print(root)
    if cnt > 5:
        break
         

datasets/CED_2/HR/CED_simple/simple_rabbits
datasets/CED_2/HR/CED_simple/simple_objects_dynamic
datasets/CED_2/HR/CED_simple/simple_color_keyboard_2
datasets/CED_2/HR/CED_simple/simple_objects
datasets/CED_2/HR/CED_simple/simple_wires_1
datasets/CED_2/HR/CED_simple/simple_color_keyboard_3


In [13]:
import os

file = open('test.txt')
lines = file.readlines()

In [14]:
lines

['0\n', '1\n', '2\n', '3\n', '4\n', '5\n', '6']

In [15]:
empty_list = [0, 1, 2]
for i in empty_list:
    del lines[i]

file.close()

In [1]:
import os
import os.path as osp
import numpy as np
import torch
from glob import glob
import time
from multiprocessing import Pool
def process_dir(list):

    events_folder = osp.join(list[0], 'events')
    images_folder = osp.join(list[0], 'images')
    empty_list = []
    event_list = sorted(glob(osp.join(events_folder, '*.txt')))
    for idx in range(len(event_list)):
        if not osp.getsize(event_list[idx]):
            empty_list.append(idx)
            os.remove(event_list[idx])
            os.remove(osp.join(images_folder, f'{(idx+1):06d}.png'))

    with open(osp.join(list[0], 'timestamp.txt'), 'r') as f:
        lines = f.readlines()
        for i in reversed(empty_list):
            del lines[i+1]
        f = open(osp.join(list[0], 'timestamp.txt'), 'w')
        f.writelines(lines)
        f.close()

    img_list = sorted(glob(osp.join(images_folder, '*.png')))
    for i in range(len(img_list)):
        old_name = img_list[i]
        new_name = osp.join(images_folder, f'{i:06d}.png')
        os.rename(old_name, new_name)

    event_list = sorted(glob(osp.join(events_folder, '*.txt')))
    for i in range(len(event_list)):
        old_name = event_list[i]
        new_name = osp.join(events_folder, f'{i:06d}.txt')
        os.rename(old_name, new_name)

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_list = ['datasets/CED_2/HR/CED_driving/driving_city_sun_2']

In [3]:
process_dir(path_list)

In [22]:
path = 'datasets/CED_2/HR/CED_driving/driving_city_sun_2'
events_folder = osp.join(path, 'events')
event_list = sorted(glob(osp.join(events_folder, '*.txt')))
for idx in range(len(event_list)):
    if not osp.getsize(event_list[idx]):
        print(idx)

In [3]:
import os

root_path = 'datasets/CED_2/HR'

pathlist = []
for root, subdir, _ in os.walk(root_path):
    if 'images' in subdir:
        pathlist.append(root)

print(len(pathlist))

84


In [4]:
pathlist[0]

'datasets/CED_2/HR/CED_simple/simple_rabbits'

In [1]:
import os
import os.path as osp
from glob import glob
import time
from multiprocessing import Pool


#input list is [''datasets/CED/HR/CED_simple/simple_rabbits', 'datasets/CED_2/LRx4/CED_simple/simple_rabbits']
def process_dir(list):
    print(list[0])

    events_folder = osp.join(list[0], 'events')
    images_folder = osp.join(list[1], 'images')
    empty_list = []
    event_list = sorted(glob(osp.join(events_folder, '*.txt')))
    for idx in range(len(event_list)):
        if not osp.getsize(event_list[idx]):
            empty_list.append(idx)
            # os.remove(event_list[idx])
            # print(f"remove empty file {event_list[idx]}")
            os.remove(osp.join(images_folder, f'{(idx+1):06d}.png'))
            print(f"remove empty file {event_list[idx]}")

    with open(osp.join(list[1], 'timestamp.txt'), 'r') as f:
        lines = f.readlines()
        for i in reversed(empty_list):
            del lines[i+1]
        f = open(osp.join(list[1], 'timestamp.txt'), 'w')
        f.writelines(lines)
        f.close()

    img_list = sorted(glob(osp.join(images_folder, '*.png')))
    for i in range(len(img_list)):
        old_name = img_list[i]
        new_name = osp.join(images_folder, f'{i:06d}.png')
        os.rename(old_name, new_name)

    # event_list = sorted(glob(osp.join(events_folder, '*.txt')))
    # for i in range(len(event_list)):
    #     old_name = event_list[i]
    #     new_name = osp.join(events_folder, f'{i:06d}.txt')
    #     os.rename(old_name, new_name)

    out_message = list[1].split('/')[-2] + '/' + list[1].split('/')[-1]
    print(f"Process {out_message} finished.")

In [ ]:
root_path = 'datasets/CED_2/LRx4'

pathlist = []
for root, subdir, _ in os.walk(root_path):
    if 'images' in subdir:
        former = osp.join('datasets/CED', root.split('/CED_2/LRx4/')[1])
        pathlist.append([former, root])

T1 = time.time()

pool = Pool(40)
pool.map(process_dir, pathlist)
pool.close()
pool.join()


In [4]:
import os.path as osp
root_path = 'datasets/CED_2/LRx4'

pathlist = []
for root, subdir, _ in os.walk(root_path):
    if 'images' in subdir:
        former = osp.join('datasets/CED/HR', root.split('/CED_2/LRx4/')[1])
        pathlist.append([former, root])

In [5]:
pathlist[0]

['datasets/CED/HR/CED_simple/simple_rabbits',
 'datasets/CED_2/LRx4/CED_simple/simple_rabbits']

In [1]:
import h5py

file = h5py.File('datasets/CED_2_h5/Voxel_3/HR/simple_fruit_fast.h5', 'r')

In [3]:
file['voxels/000000'].shape

(3, 260, 346)

# make meta_info CED_2 

In [4]:
import glob
import os.path as osp

root_path = 'datasets/CED_3/HR'
wr_file = '/home/kaidc/code/e2vsr/basicsr/data/meta_info/mata_info_CED_3_h5_train.txt'

with open(wr_file, 'w') as f:
    for root, subdir, _ in os.walk(root_path):
        if 'images' in subdir:
            length = str(len(glob.glob(osp.join(root, 'images', '*.png'))))
            wr_str = osp.basename(root) + '.h5' + ' ' + length + '\n'
            f.write(wr_str)

f.close()




In [5]:
import h5py
import numpy as np

file_path = 'datasets/CED_3_h5/Voxel_3/HR/indoors_foosball_1.h5'

file = h5py.File(file_path, 'r')

In [8]:
np.min(file['voxels/000001'][:])

-1.0

In [ ]:
for i in range(file.attrs['num_events']):
    if np.min(file[f'voxels/{i:06d}'][:]) < 0:
        print(f"{i:06d}")

In [3]:
np.all(temp == 0)

False

In [4]:
np.isnan(temp).any()

False

In [4]:
from basicsr.utils.hdf5_util import hdf5_packager
test_h5 = hdf5_packager('./temp.h5')

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
txt_file = 'datasets/CED/HR/CED_additional_IR_filter/outdoor_shadow_1_infrared/events/000000.txt'

In [6]:
test_h5.package_voxel(txt_file, 3)

tensor(0, dtype=torch.int16)


In [7]:
import h5py
file_2 = h5py.File('./temp.h5', 'r')

In [47]:
file_2.keys()

<KeysViewHDF5 ['voxels']>

In [48]:
file_2['voxels'].keys()

<KeysViewHDF5 ['000144']>

In [52]:
v = file_2['voxels/000144'][:]

In [53]:
np.max(v)

nan

In [55]:
import pandas as pd
data = pd.read_csv(txt_file, delim_whitespace=True, header=None, names=['t', 'x', 'y', 'p'], dtype={'t': np.float64, 'x': np.int16, 'y': np.int16, 'p': np.int16})

In [56]:
data

,t,x,y,p
0,1.555342e+09,156,217,1


In [58]:
import torch
t = torch.from_numpy(data['t'].values)
x = torch.from_numpy(data['x'].values)
y = torch.from_numpy(data['y'].values)
p = torch.from_numpy(data['p'].values)

In [74]:
from basicsr.utils.event_utils import *
bins = 3
print(x, y, t, p, bins)
voxel = events_to_voxel_torch(x, y, t, p, bins, device=None, sensor_size=(260, 346))

tensor([156], dtype=torch.int16) tensor([217], dtype=torch.int16) tensor([1.5553e+09], dtype=torch.float64) tensor([1], dtype=torch.int16) 3


In [76]:
dt = t[-1]-t[0]
print(dt)

tensor(0., dtype=torch.float64)


In [77]:
B = 3
t_norm = (t-t[0])/dt*(B-1)

In [78]:
t_norm

tensor([nan], dtype=torch.float64)

In [1]:
from distutils.log import error
import os
import os.path as osp
from glob import glob
import time
from multiprocessing import Pool


#input list is [''datasets/CED_3/HR/CED_simple/simple_rabbits']
def process_dir(list):
    print(list[0])

    events_folder = osp.join(list[0], 'events')
    images_folder = osp.join(list[0], 'images')
    error_list = []
    event_list = sorted(glob(osp.join(events_folder, '*.txt')))

    for idx in range(len(event_list)):
        with open(event_list[idx], 'r') as f:
            lines = f.readlines()
            if len(lines) == 1:

                if idx == (len(event_list) - 1):
                    error_list.append(idx)
                    os.remove(event_list[idx])
                    os.remove(osp.join(images_folder, f'{(idx+1):06d}.png'))
                    print("error the last one idx: ", idx)
                    print(f"remove event file {event_list[idx]}")
                    break

                with open(event_list[idx+1], 'r+') as f2:
                    content = f2.read()
                    f2.seek(0, 0)
                    f2.write(lines[0]+content)

                error_list.append(idx)
                print("error one idx: ", idx)

                os.remove(event_list[idx])
                os.remove(osp.join(images_folder, f'{(idx+1):06d}.png'))
                print(f"remove event file {event_list[idx]}")

            elif len(lines) == 0:
                error_list.append(idx)
                os.remove(event_list[idx])
                print(f"remove empty file {event_list[idx]}")
                os.remove(osp.join(images_folder, f'{(idx+1):06d}.png'))

            else:
                continue

    with open(osp.join(list[0], 'timestamp.txt'), 'r') as f:
        lines = f.readlines()
        for i in reversed(sorted(error_list)):
            del lines[i+1]

    f.close()

    f = open(osp.join(list[0], 'timestamp.txt'), 'w')
    f.writelines(lines)
    f.close()

    img_list = sorted(glob(osp.join(images_folder, '*.png')))
    for i in range(len(img_list)):
        old_name = img_list[i]
        new_name = osp.join(images_folder, f'{i:06d}.png')
        os.rename(old_name, new_name)

    event_list = sorted(glob(osp.join(events_folder, '*.txt')))
    for i in range(len(event_list)):
        old_name = event_list[i]
        new_name = osp.join(events_folder, f'{i:06d}.txt')
        os.rename(old_name, new_name)

    out_message = list[0].split('/')[-2] + '/' + list[0].split('/')[-1]
    print(f"Process {out_message} finished.")

In [2]:
process_dir(['datasets/CED_3/HR/CED_additional_IR_filter/outdoor_shadow_1_infrared'])

datasets/CED_3/HR/CED_additional_IR_filter/outdoor_shadow_1_infrared
error one idx:  144
remove event file datasets/CED_3/HR/CED_additional_IR_filter/outdoor_shadow_1_infrared/events/000144.txt
remove empty file datasets/CED_3/HR/CED_additional_IR_filter/outdoor_shadow_1_infrared/events/000161.txt
error one idx:  165
remove event file datasets/CED_3/HR/CED_additional_IR_filter/outdoor_shadow_1_infrared/events/000165.txt
error the last one idx:  573
remove event file datasets/CED_3/HR/CED_additional_IR_filter/outdoor_shadow_1_infrared/events/000573.txt
Process CED_additional_IR_filter/outdoor_shadow_1_infrared finished.


In [11]:
file = 'datasets/CED_3/HR/CED_driving/driving_city_sun_2/events/001765.txt'
with open(file, 'r') as f:
    lines = f.readlines()

In [3]:
root_path = 'datasets/CED_3/HR'

pathlist = []
for root, subdir, _ in os.walk(root_path):
    if 'images' in subdir:
        # former = osp.join('datasets/CED/HR', root.split('/CED_2/LRx2/')[1])
        pathlist.append([root])

In [6]:
pathlist

[['datasets/CED_3/HR/CED_simple/simple_rabbits'],
 ['datasets/CED_3/HR/CED_simple/simple_objects_dynamic'],
 ['datasets/CED_3/HR/CED_simple/simple_color_keyboard_2'],
 ['datasets/CED_3/HR/CED_simple/simple_objects'],
 ['datasets/CED_3/HR/CED_simple/simple_wires_1'],
 ['datasets/CED_3/HR/CED_simple/simple_color_keyboard_3'],
 ['datasets/CED_3/HR/CED_simple/simple_jenga_1'],
 ['datasets/CED_3/HR/CED_simple/simple_color_keyboard_1'],
 ['datasets/CED_3/HR/CED_simple/simple_carpet'],
 ['datasets/CED_3/HR/CED_simple/simple_fruit_fast'],
 ['datasets/CED_3/HR/CED_simple/simple_carpet_fast'],
 ['datasets/CED_3/HR/CED_simple/simple_flowers_infrared'],
 ['datasets/CED_3/HR/CED_simple/simple_wires_2'],
 ['datasets/CED_3/HR/CED_simple/simple_jenga_2'],
 ['datasets/CED_3/HR/CED_simple/simple_jenga_destroy'],
 ['datasets/CED_3/HR/CED_simple/simple_fruit'],
 ['datasets/CED_3/HR/CED_indoors/indoors_foosball_1'],
 ['datasets/CED_3/HR/CED_indoors/indoors_window_autoexposure'],
 ['datasets/CED_3/HR/CED_in

In [14]:
from distutils.log import error
import os
import os.path as osp
from glob import glob
import time
from multiprocessing import Pool


#input list is [''datasets/CED_3/HR/CED_simple/simple_rabbits']
def process_dir(list):
    # print(list[0])

    events_folder = osp.join(list[0], 'events')
    images_folder = osp.join(list[0], 'images')
    # error_list = []
    event_list = sorted(glob(osp.join(events_folder, '*.txt')))
    img_list = sorted(glob(osp.join(images_folder, "*.png")))
    f = open(osp.join(list[0], 'timestamp.txt'), 'r')
    lines = f.readlines()
    assert len(event_list) == len(img_list) - 1 == len(lines) - 1, f"check foloder"

    for idx in range(len(event_list)):
        with open(event_list[idx], 'r') as f:
            lines = f.readlines()
            if len(lines) == 1:

                # if idx == (len(event_list) - 1):
                #     error_list.append(idx)
                #     os.remove(event_list[idx])
                #     os.remove(osp.join(images_folder, f'{(idx+1):06d}.png'))
                #     print("error the last one idx: ", idx)
                #     print(f"remove event file {event_list[idx]}")
                #     break

                # with open(event_list[idx+1], 'r+') as f2:
                    # content = f2.read()
                    # f2.seek(0, 0)
                    # f2.write(lines[0]+content)

                # error_list.append(idx)
                print("error one idx: ", event_list[idx])

                # os.remove(event_list[idx])
                # os.remove(osp.join(images_folder, f'{(idx+1):06d}.png'))
                # print(f"remove event file {event_list[idx]}")

            elif len(lines) == 0:
                # error_list.append(idx)
                # os.remove(event_list[idx])
                print("empty one idx: ", event_list[idx])
                # print(f"remove empty file {event_list[idx]}")
                # os.remove(osp.join(images_folder, f'{(idx+1):06d}.png'))

            else:
                continue

    # with open(osp.join(list[0], 'timestamp.txt'), 'r') as f:
    #     lines = f.readlines()
    #     for i in reversed(sorted(error_list)):
    #         del lines[i+1]

    # f.close()

    # f = open(osp.join(list[0], 'timestamp.txt'), 'w')
    # f.writelines(lines)
    # f.close()

    # img_list = sorted(glob(osp.join(images_folder, '*.png')))
    # for i in range(len(img_list)):
    #     old_name = img_list[i]
    #     new_name = osp.join(images_folder, f'{i:06d}.png')
    #     os.rename(old_name, new_name)

    # event_list = sorted(glob(osp.join(events_folder, '*.txt')))
    # for i in range(len(event_list)):
    #     old_name = event_list[i]
    #     new_name = osp.join(events_folder, f'{i:06d}.txt')
    #     os.rename(old_name, new_name)

    # out_message = list[0].split('/')[-2] + '/' + list[0].split('/')[-1]
    # print(f"Process {out_message} finished.")

In [15]:
root_path = 'datasets/CED_3/HR'

pathlist = []
for root, subdir, _ in os.walk(root_path):
    if 'images' in subdir:
        # former = osp.join('datasets/CED/HR', root.split('/CED_2/LRx2/')[1])
        pathlist.append([root])

In [16]:
pool = Pool(40)
pool.map(process_dir, pathlist)
pool.close()
pool.join()

In [6]:
import os
import os.path as osp

root_path = 'datasets/CED_3/LRx4'

pathlist = []
for root, subdir, _ in os.walk(root_path):
    if 'images' in subdir:
        former = osp.join('datasets/CED/HR', root.split('/CED_3/LRx4/')[1])
        pathlist.append([former, root])


In [7]:
pathlist[0]

['datasets/CED/HR/CED_simple/simple_rabbits',
 'datasets/CED_3/LRx4/CED_simple/simple_rabbits']

In [1]:
def change_0_to_neg(txt_file):
    to_write_list = []
    with open(txt_file, 'r') as f:
        for line in f.readlines():
            line = line.strip('\n').split(' ')
            if line[-1] == '0':
                line[-1] = '-1'
            to_write = line[0] + ' ' + line[1]+ ' ' + line[2] + ' ' + line[-1] + '\n'
            to_write_list.append(to_write)
    f.close()

    f = open(txt_file, 'w')
    f.writelines(to_write_list)

In [2]:
txt_file = '000000.txt'
change_0_to_neg(txt_file)

In [28]:
import h5py
file = h5py.File('1-4-f-200.h5', 'r')

In [29]:
file.keys()

<KeysViewHDF5 ['images', 'masks', 'sharp_images', 'voxels']>

In [42]:
EF_im = file['images/image000000000'][:]

In [33]:
EF_vo = file['voxels/voxel000000000'][:]

In [44]:
np.max(EF_im)

232

In [9]:
from distutils.log import error
import os
import os.path as osp
from glob import glob
import time
from multiprocessing import Pool
root_path = 'datasets/CED_3/HR'

dir_list = []
for root, subdir, _ in os.walk(root_path):
    if 'events' in subdir:
        dir_list.append([osp.join(root, 'events')])

In [10]:
dir_list[0]

['datasets/CED_3/HR/CED_simple/simple_rabbits/events']

In [11]:
def process_dir(list):
    event_list = sorted(glob(osp.join(list[0], '*.txt')))
    for txt_file in event_list:
        change_0_to_neg(txt_file)

    out_message = list[0].split('/')[-3] + '/' + list[0].split('/')[-1]
    print(f"Process {out_message} finished.")

def change_0_to_neg(txt_file):
    to_write_list = []
    with open(txt_file, 'r') as f:
        for line in f.readlines():
            line = line.strip('\n').split(' ')
            if line[-1] == '0':
                line[-1] = '-1'
            to_write = line[0] + ' ' + line[1]+ ' ' + line[2] + ' ' + line[-1] + '\n'
            to_write_list.append(to_write)
    f.close()

    f = open(txt_file, 'w')
    f.writelines(to_write_list)
    f.close()


In [12]:
print(dir_list[0])
process_dir(dir_list[0])

['datasets/CED_3/HR/CED_simple/simple_rabbits/events']
Process CED_simple/events finished.


In [11]:
import os
import os.path as osp

folder_path = 'datasets/CED_2/HR'
root_path_list = []
h5_path_list = []
for root, subdir, _ in os.walk(folder_path):
    if 'images' in subdir:
        root_path_list.append(root)
        out = osp.join('datasets/CED_2_h5/Voxel_3/HR', osp.basename(root) + '.h5')
        h5_path_list.append(out)

In [12]:
root_path_list[0]

'datasets/CED_2/HR/CED_simple/simple_rabbits'

In [13]:
h5_path_list[0]

'datasets/CED_3_h5/Voxel_3/HR/simple_rabbits.h5'

In [17]:
from basicsr.utils.hdf5_util import *

test_list = ['datasets/CED_2/HR/CED_simple/simple_rabbits', 'datasets/CED_3_h5/Voxel_3/HR/driving_tunnel.h5']

write_h5_worker(test_list)

In [1]:
import h5py

file = h5py.File('datasets/CED_3_h5/Voxel_3/HR/driving_tunnel.h5', 'r')

In [2]:
np_arr = file['voxels/000000'][:]

In [5]:
import numpy as np

print(np.min(np_arr))
print(np.max(np_arr))

-2.0646965503692627
1.2698324918746948


In [1]:
from basicsr.utils.hdf5_util import hdf5_packager

file = hdf5_packager('./temp.h5')

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
txt_file = 'datasets/CED_3/HR/CED_driving/driving_tunnel/events/000000.txt'

In [3]:
file.package_voxel(txt_file, 3)

# vimeo&vid4 process

In [2]:
import os.path as osp
import os
from glob import glob

In [12]:
## meta_info_vimeo_h5_train.txt
train_txt = 'datasets/vimeo_septuplet_h5/sep_trainlist.txt'
meta_file = 'basicsr/data/meta_info/meta_info_vimeo_h5_train.txt'

with open(train_txt, 'r') as f1:
    line_list = f1.read().splitlines()
f1.close()

with open(meta_file, 'w') as f2:
    for line in line_list:
        to_wrt = line.replace('/', '_') + '.h5' + ' 13' + '\n'
        f2.write(to_wrt)
f2.close()

In [ ]:
## meta_info_vimeo_h5_test.txt
test_txt = 'datasets/vimeo_septuplet_h5/sep_testlist.txt'
meta_file = 'basicsr/data/meta_info/meta_info_vimeo_h5_test.txt'

with open(test_txt, 'r') as f1:
    line_list = f1.read().splitlines()
f1.close()

with open(meta_file, 'w') as f2:
    for line in line_list:
        to_wrt = line.replace('/', '_') + '.h5' + ' 7' + '\n'
        f2.write(to_wrt)
f2.close()